# 📚 Library Management System (LMS)

## 📝 Introduction
This notebook demonstrates the design and implementation of a **Library Management System (LMS)** using **SQLite**, **Python**, and **Jupyter Widgets**.  
The project simulates real-world library operations such as **book cataloging, member registration, borrowing, returning, and reservations**.  

The system is designed to:
- Manage **members, authors, publishers, books, and book copies**.  
- Track **loans, reservations, and book availability**.  
- Provide **interactive widgets** for adding members and searching for books.  
- Offer **basic statistics** on library usage (e.g., number of members, books, available copies, and active loans).  

By the end of this notebook, you will have a fully functioning **relational database schema** with core operations that model how a real library system works.  

---


In [1]:
# Importing rEquired Libraries 

import sqlite3
import pandas as pd
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, Markdown

In [2]:
# Create or connect to SQLite database
def create_database():
    conn = sqlite3.connect('library_management.sqlite')
    cursor = conn.cursor()
    
    # Enable foreign keys
    cursor.execute("PRAGMA foreign_keys = ON")

###  Create Tables

In [3]:
# Create or connect to SQLite database
def create_database():
    conn = sqlite3.connect('library_management.sqlite')
    cursor = conn.cursor()
    
    # Enable foreign keys
    cursor.execute("PRAGMA foreign_keys = ON")
    
    # Create Tables
    tables_sql = [
        # Members Table
        """
        CREATE TABLE IF NOT EXISTS Members (
            member_id INTEGER PRIMARY KEY AUTOINCREMENT,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            phone TEXT,
            address TEXT,
            membership_date TEXT NOT NULL,
            membership_status TEXT DEFAULT 'Active' CHECK(membership_status IN ('Active', 'Suspended', 'Expired')),
            created_at TEXT DEFAULT CURRENT_TIMESTAMP,
            updated_at TEXT DEFAULT CURRENT_TIMESTAMP
        )
        """,
        
        # Authors Table
        """
        CREATE TABLE IF NOT EXISTS Authors (
            author_id INTEGER PRIMARY KEY AUTOINCREMENT,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            birth_date TEXT,
            death_date TEXT,
            nationality TEXT,
            biography TEXT,
            created_at TEXT DEFAULT CURRENT_TIMESTAMP
        )
        """,
        
        # Publishers Table
        """
        CREATE TABLE IF NOT EXISTS Publishers (
            publisher_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL UNIQUE,
            address TEXT,
            phone TEXT,
            email TEXT,
            website TEXT,
            created_at TEXT DEFAULT CURRENT_TIMESTAMP
        )
        """,
        
        # Books Table
        """
        CREATE TABLE IF NOT EXISTS Books (
            book_id INTEGER PRIMARY KEY AUTOINCREMENT,
            isbn TEXT UNIQUE NOT NULL,
            title TEXT NOT NULL,
            publisher_id INTEGER NOT NULL,
            publication_year TEXT,
            edition INTEGER DEFAULT 1,
            language TEXT DEFAULT 'English',
            pages INTEGER,
            description TEXT,
            category TEXT,
            price REAL,
            created_at TEXT DEFAULT CURRENT_TIMESTAMP,
            updated_at TEXT DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (publisher_id) REFERENCES Publishers(publisher_id) ON DELETE CASCADE
        )
        """,
        
        # Book_Authors Table (Many-to-Many)
        """
        CREATE TABLE IF NOT EXISTS Book_Authors (
            book_id INTEGER NOT NULL,
            author_id INTEGER NOT NULL,
            PRIMARY KEY (book_id, author_id),
            FOREIGN KEY (book_id) REFERENCES Books(book_id) ON DELETE CASCADE,
            FOREIGN KEY (author_id) REFERENCES Authors(author_id) ON DELETE CASCADE
        )
        """,
        
        # Book_Copies Table
        """
        CREATE TABLE IF NOT EXISTS Book_Copies (
            copy_id INTEGER PRIMARY KEY AUTOINCREMENT,
            book_id INTEGER NOT NULL,
            copy_number INTEGER NOT NULL,
            status TEXT DEFAULT 'Available' CHECK(status IN ('Available', 'Borrowed', 'Reserved', 'Maintenance', 'Lost')),
            acquisition_date TEXT,
            condition TEXT DEFAULT 'Good' CHECK(condition IN ('New', 'Good', 'Fair', 'Poor')),
            location TEXT,
            created_at TEXT DEFAULT CURRENT_TIMESTAMP,
            UNIQUE(book_id, copy_number),
            FOREIGN KEY (book_id) REFERENCES Books(book_id) ON DELETE CASCADE
        )
        """,
        
        # Loans Table
        """
        CREATE TABLE IF NOT EXISTS Loans (
            loan_id INTEGER PRIMARY KEY AUTOINCREMENT,
            member_id INTEGER NOT NULL,
            copy_id INTEGER NOT NULL,
            loan_date TEXT NOT NULL,
            due_date TEXT NOT NULL,
            return_date TEXT,
            status TEXT DEFAULT 'Active' CHECK(status IN ('Active', 'Returned', 'Overdue')),
            fine_amount REAL DEFAULT 0.00,
            created_at TEXT DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (member_id) REFERENCES Members(member_id) ON DELETE CASCADE,
            FOREIGN KEY (copy_id) REFERENCES Book_Copies(copy_id) ON DELETE CASCADE,
            CHECK (due_date > loan_date),
            CHECK (return_date IS NULL OR return_date >= loan_date)
        )
        """,
        
        # Reservations Table
        """
        CREATE TABLE IF NOT EXISTS Reservations (
            reservation_id INTEGER PRIMARY KEY AUTOINCREMENT,
            member_id INTEGER NOT NULL,
            book_id INTEGER NOT NULL,
            reservation_date TEXT NOT NULL,
            status TEXT DEFAULT 'Pending' CHECK(status IN ('Pending', 'Fulfilled', 'Cancelled')),
            priority INTEGER DEFAULT 1,
            created_at TEXT DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (member_id) REFERENCES Members(member_id) ON DELETE CASCADE,
            FOREIGN KEY (book_id) REFERENCES Books(book_id) ON DELETE CASCADE
        )
        """
    ]
    
    for sql in tables_sql:
        cursor.execute(sql)
    
    conn.commit()
    return conn

In [4]:
# Initialize database
conn = create_database()
cursor = conn.cursor()


### Inserting Sample Data

In [5]:
def insert_sample_data_basic():
    try:
        # Clear all data first (order matters: child → parent)
        cursor.execute("DELETE FROM Book_Authors")
        cursor.execute("DELETE FROM Book_Copies")
        cursor.execute("DELETE FROM Books")
        cursor.execute("DELETE FROM Authors")
        cursor.execute("DELETE FROM Publishers")
        cursor.execute("DELETE FROM Members")
        
        # Reset autoincrement counters
        cursor.execute("DELETE FROM sqlite_sequence WHERE name IN ('Publishers','Authors','Books','Book_Copies','Book_Authors','Members')")

        # Insert Publishers
        cursor.execute("INSERT INTO Publishers (name, address, phone, email, website) VALUES (?, ?, ?, ?, ?)", 
                      ('Penguin Random House', '123 Book Ave, New York, NY', '555-0101', 'info@penguin.com', 'www.penguin.com'))
        penguin_id = cursor.lastrowid
        
        cursor.execute("INSERT INTO Publishers (name, address, phone, email, website) VALUES (?, ?, ?, ?, ?)", 
                      ('HarperCollins', '456 Publisher St, London, UK', '555-0102', 'contact@harpercollins.com', 'www.harpercollins.com'))
        harper_id = cursor.lastrowid
        
        # Insert Authors
        cursor.execute("INSERT INTO Authors (first_name, last_name, birth_date, death_date, nationality, biography) VALUES (?, ?, ?, ?, ?, ?)", 
                      ('George', 'Orwell', '1903-06-25', None, 'British', 'Author of 1984 and Animal Farm'))
        orwell_id = cursor.lastrowid
        
        cursor.execute("INSERT INTO Authors (first_name, last_name, birth_date, death_date, nationality, biography) VALUES (?, ?, ?, ?, ?, ?)", 
                      ('J.K.', 'Rowling', '1965-07-31', None, 'British', 'Author of Harry Potter series'))
        rowling_id = cursor.lastrowid
        
        cursor.execute("INSERT INTO Authors (first_name, last_name, birth_date, death_date, nationality, biography) VALUES (?, ?, ?, ?, ?, ?)", 
                      ('Stephen', 'King', '1947-09-21', None, 'American', 'Master of horror fiction'))
        king_id = cursor.lastrowid
        
        # Insert Books (using real publisher IDs)
        cursor.execute("INSERT INTO Books (isbn, title, publisher_id, publication_year, edition, language, pages, description, category, price) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                      ('978-0451524935', '1984', penguin_id, '1949', 1, 'English', 328, 'Dystopian novel', 'Fiction', 12.99))
        book1984_id = cursor.lastrowid
        
        cursor.execute("INSERT INTO Books (isbn, title, publisher_id, publication_year, edition, language, pages, description, category, price) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                      ('978-0439064873', 'Harry Potter and the Chamber of Secrets', harper_id, '1998', 1, 'English', 341, 'Second book in Harry Potter series', 'Fantasy', 15.99))
        harry_id = cursor.lastrowid
        
        cursor.execute("INSERT INTO Books (isbn, title, publisher_id, publication_year, edition, language, pages, description, category, price) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                      ('978-1501142970', 'The Shining', penguin_id, '1977', 1, 'English', 447, 'Horror novel about a haunted hotel', 'Horror', 14.99))
        shining_id = cursor.lastrowid
        
        # Insert Book_Authors (using fetched IDs)
        cursor.execute("INSERT INTO Book_Authors (book_id, author_id) VALUES (?, ?)", (book1984_id, orwell_id))
        cursor.execute("INSERT INTO Book_Authors (book_id, author_id) VALUES (?, ?)", (harry_id, rowling_id))
        cursor.execute("INSERT INTO Book_Authors (book_id, author_id) VALUES (?, ?)", (shining_id, king_id))
        
        # Insert Book_Copies
        cursor.execute("INSERT INTO Book_Copies (book_id, copy_number, status, acquisition_date, condition, location) VALUES (?, ?, ?, ?, ?, ?)", 
                      (book1984_id, 1, 'Available', '2023-01-15', 'Good', 'Shelf A1'))
        cursor.execute("INSERT INTO Book_Copies (book_id, copy_number, status, acquisition_date, condition, location) VALUES (?, ?, ?, ?, ?, ?)", 
                      (book1984_id, 2, 'Available', '2023-01-15', 'New', 'Shelf A1'))
        cursor.execute("INSERT INTO Book_Copies (book_id, copy_number, status, acquisition_date, condition, location) VALUES (?, ?, ?, ?, ?, ?)", 
                      (harry_id, 1, 'Available', '2023-02-20', 'Good', 'Shelf B2'))
        cursor.execute("INSERT INTO Book_Copies (book_id, copy_number, status, acquisition_date, condition, location) VALUES (?, ?, ?, ?, ?, ?)", 
                      (shining_id, 1, 'Available', '2023-03-10', 'Fair', 'Shelf C3'))
        
        # Insert Members
        cursor.execute("INSERT INTO Members (first_name, last_name, email, phone, address, membership_date, membership_status) VALUES (?, ?, ?, ?, ?, ?, ?)", 
                      ('John', 'Doe', 'john.doe@email.com', '555-1234', '123 Main St', '2023-01-01', 'Active'))
        cursor.execute("INSERT INTO Members (first_name, last_name, email, phone, address, membership_date, membership_status) VALUES (?, ?, ?, ?, ?, ?, ?)", 
                      ('Jane', 'Smith', 'jane.smith@email.com', '555-5678', '456 Oak Ave', '2023-02-15', 'Active'))
        
        conn.commit()
        print("✅ Sample data inserted successfully!")
        
    except Exception as e:
        print(f"❌ Error inserting sample data: {e}")
        conn.rollback()


In [6]:
# Utility Functions
def execute_query(query, params=None):
    """Execute SQL query and return results as DataFrame"""
    if params:
        df = pd.read_sql_query(query, conn, params=params)
    else:
        df = pd.read_sql_query(query, conn)
    return df

def display_table(table_name, limit=10):
    """Display a table with nice formatting"""
    df = execute_query(f"SELECT * FROM {table_name} LIMIT {limit}")
    display(Markdown(f"### {table_name} Table"))
    display(df)

In [7]:
# Interactive Widgets for Library Operations
def add_member(first_name, last_name, email, phone, address):
    """Add a new member to the library"""
    try:
        cursor.execute("""
            INSERT INTO Members (first_name, last_name, email, phone, address, membership_date)
            VALUES (?, ?, ?, ?, ?, date('now'))
        """, (first_name, last_name, email, phone, address))
        conn.commit()
        print("✅ Member added successfully!")
    except sqlite3.IntegrityError:
        print("❌ Error: Email already exists!")

def search_books(title=None, author=None, category=None):
    """Search for books by title, author, or category"""
    query = """
        SELECT b.book_id, b.title, b.isbn, a.first_name || ' ' || a.last_name as author, 
               b.category, b.publication_year, bc.status
        FROM Books b
        JOIN Book_Authors ba ON b.book_id = ba.book_id
        JOIN Authors a ON ba.author_id = a.author_id
        JOIN Book_Copies bc ON b.book_id = bc.book_id
        WHERE 1=1
    """
    params = []
    
    if title:
        query += " AND b.title LIKE ?"
        params.append(f'%{title}%')
    if author:
        query += " AND (a.first_name LIKE ? OR a.last_name LIKE ?)"
        params.extend([f'%{author}%', f'%{author}%'])
    if category:
        query += " AND b.category = ?"
        params.append(category)
    
    query += " GROUP BY b.book_id"
    
    df = execute_query(query, params)
    return df


In [8]:
# Display Database Schema
display(Markdown("# 📚 Library Management System Database"))
display(Markdown("## Database Schema Overview"))

# Show all tables
tables = execute_query("SELECT name FROM sqlite_master WHERE type='table'")
display(Markdown("### Available Tables:"))
for table in tables['name']:
    display(Markdown(f"- **{table}**"))

# Display sample data from each table
for table in tables['name']:
    display_table(table)


# 📚 Library Management System Database

## Database Schema Overview

### Available Tables:

- **Members**

- **sqlite_sequence**

...

,loan_id,member_id,copy_id,loan_date,due_date,return_date,status,fine_amount,created_at


### Reservations Table

,reservation_id,member_id,book_id,reservation_date,status,priority,created_at


### Fines Table

,fine_id,loan_id,member_id,fine_amount,amount_paid,payment_date


In [9]:
# Example: Search Books
display(Markdown("## 🔍 Book Search Example"))
search_results = search_books(title="Harry")
display(search_results)

# Close connection when done
def close_connection():
    conn.close()
    print("Database connection closed.")


## 🔍 Book Search Example

,book_id,title,isbn,author,category,publication_year,status
0,2,Harry Potter and the Chamber of Secrets,978-0439064873,J.K. Rowling,Fantasy,1998,Available


In [10]:
# Interactive Widget Example
display(Markdown("## 👥 Add New Member (Interactive)"))
first_name = widgets.Text(description="First Name:")
last_name = widgets.Text(description="Last Name:")
email = widgets.Text(description="Email:")
phone = widgets.Text(description="Phone:")
address = widgets.Text(description="Address:")
add_button = widgets.Button(description="Add Member")

def on_add_button_clicked(b):
    add_member(first_name.value, last_name.value, email.value, phone.value, address.value)
    display_table("Members")

add_button.on_click(on_add_button_clicked)

display(widgets.VBox([first_name, last_name, email, phone, address, add_button]))


## 👥 Add New Member (Interactive)

In [11]:
# Statistics Dashboard
display(Markdown("## 📊 Library Statistics"))
stats_query = """
    SELECT 
        (SELECT COUNT(*) FROM Members) as total_members,
        (SELECT COUNT(*) FROM Books) as total_books,
        (SELECT COUNT(*) FROM Book_Copies WHERE status = 'Available') as available_copies,
        (SELECT COUNT(*) FROM Loans WHERE status = 'Active') as active_loans
"""
stats = execute_query(stats_query)
display(stats)


## 📊 Library Statistics

,total_members,total_books,available_copies,active_loans
0,4,3,4,0


In [12]:
# Don't forget to close connection when notebook is closed
import atexit
atexit.register(close_connection)

<function __main__.close_connection()>

### Adding Fines table , borrow and return functions

In [13]:

# Connect to database
conn = sqlite3.connect("library_management.sqlite")
cursor = conn.cursor()

# --- Create Fines table ---
cursor.execute("""
CREATE TABLE IF NOT EXISTS Fines (
    fine_id INTEGER PRIMARY KEY AUTOINCREMENT,
    loan_id INTEGER,
    member_id INTEGER,
    fine_amount REAL,
    amount_paid REAL,
    payment_date TEXT,
    FOREIGN KEY (loan_id) REFERENCES Loans(loan_id),
    FOREIGN KEY (member_id) REFERENCES Members(member_id)
)
""")
conn.commit()

# --- Borrow Book ---
def borrow_book(member_id, copy_id, loan_days=14):
    """Borrow a book if available"""
    loan_date = datetime.now().strftime('%Y-%m-%d')
    due_date = (datetime.now() + timedelta(days=loan_days)).strftime('%Y-%m-%d')
    
    try:
        cursor.execute("SELECT status FROM Book_Copies WHERE copy_id = ?", (copy_id,))
        result = cursor.fetchone()
        if not result:
            return {"status": False, "message": "Copy ID not found"}
        
        if result[0] != 'Available':
            return {"status": False, "message": "Book is not available"}
        
        cursor.execute("""
            INSERT INTO Loans (member_id, copy_id, loan_date, due_date, status)
            VALUES (?, ?, ?, ?, 'Borrowed')
        """, (member_id, copy_id, loan_date, due_date))
        
        cursor.execute("UPDATE Book_Copies SET status = 'Borrowed' WHERE copy_id = ?", (copy_id,))
        conn.commit()
        return {"status": True, "message": "Book borrowed successfully"}
    
    except Exception as e:
        conn.rollback()
        return {"status": False, "message": str(e)}

# --- Return Book ---
def return_book(loan_id):
    """Return a borrowed book and calculate fines"""
    return_date = datetime.now().strftime('%Y-%m-%d')
    
    try:
        cursor.execute("SELECT copy_id, due_date FROM Loans WHERE loan_id = ?", (loan_id,))
        result = cursor.fetchone()
        if not result:
            return {"status": False, "message": "Loan ID not found"}
        
        copy_id, due_date = result
        overdue_days = (datetime.now() - datetime.strptime(due_date, '%Y-%m-%d')).days
        fine = max(0, overdue_days * 10)  # Example: 10 per day fine
        
        cursor.execute("""
            UPDATE Loans 
            SET return_date = ?, status = 'Returned' 
            WHERE loan_id = ?
        """, (return_date, loan_id))
        
        cursor.execute("UPDATE Book_Copies SET status = 'Available' WHERE copy_id = ?", (copy_id,))
        conn.commit()
        
        return {
            "status": True,
            "message": "Book returned successfully",
            "overdue_days": overdue_days if overdue_days > 0 else 0,
            "fine": fine
        }
    
    except Exception as e:
        conn.rollback()
        return {"status": False, "message": str(e)}

# --- Pay Fine ---
def pay_fine(loan_id, amount_paid):
    """Pay fine for a specific loan"""
    try:
        cursor.execute("SELECT member_id, due_date, return_date FROM Loans WHERE loan_id = ?", (loan_id,))
        result = cursor.fetchone()
        if not result:
            return {"status": False, "message": "Loan ID not found"}
        
        member_id, due_date, return_date = result
        
        if not return_date:
            return {"status": False, "message": "Book not yet returned, no fine to pay"}
        
        overdue_days = (datetime.strptime(return_date, '%Y-%m-%d') - datetime.strptime(due_date, '%Y-%m-%d')).days
        fine_due = max(0, overdue_days * 10)
        
        if fine_due == 0:
            return {"status": True, "message": "No fine to pay"}
        
        if amount_paid < fine_due:
            return {"status": False, "message": f"Insufficient payment. Fine due: {fine_due}"}
        
        cursor.execute("""
            INSERT INTO Fines (loan_id, member_id, fine_amount, amount_paid, payment_date)
            VALUES (?, ?, ?, ?, ?)
        """, (loan_id, member_id, fine_due, amount_paid, datetime.now().strftime('%Y-%m-%d')))
        
        conn.commit()
        return {"status": True, "message": "Fine paid successfully", "fine_due": fine_due, "amount_paid": amount_paid}
    
    except Exception as e:
        conn.rollback()
        return {"status": False, "message": str(e)}

# --- Example Usage ---
# borrow_book(1, 1)         # Borrow a book
# return_info = return_book(1)   # Return it
# print(return_info)
# if return_info.get("fine", 0) > 0:
#     pay_info = pay_fine(1, return_info["fine"])
#     print(pay_info)


### View Member fines

In [14]:
def view_unpaid_fines(member_id):
    """View all unpaid fines for a specific member"""
    try:
        cursor.execute("""
        SELECT L.loan_id, L.due_date, L.return_date,
               (julianday(L.return_date) - julianday(L.due_date)) * 10 AS fine_due,
               IFNULL(F.amount_paid, 0) as paid
        FROM Loans L
        LEFT JOIN Fines F ON L.loan_id = F.loan_id
        WHERE L.member_id = ?
        """, (member_id,))
        
        rows = cursor.fetchall()
        fines_list = []
        
        for row in rows:
            loan_id, due_date, return_date, fine_due, paid = row
            fine_due = max(0, int(fine_due) if fine_due else 0)  # ensure non-negative
            balance = fine_due - paid
            if balance > 0:
                fines_list.append({
                    "loan_id": loan_id,
                    "due_date": due_date,
                    "return_date": return_date,
                    "fine_due": fine_due,
                    "paid": paid,
                    "balance": balance
                })
        
        if not fines_list:
            return {"status": True, "message": "No unpaid fines", "fines": []}
        return {"status": True, "message": "Unpaid fines found", "fines": fines_list}
    
    except Exception as e:
        return {"status": False, "message": str(e)}


### Example Usage 

In [15]:
# Step 1: Borrow a book (simulate)
borrow_book(1, 1)

# Step 2: Force the loan to be overdue for testing
cursor.execute("UPDATE Loans SET due_date = '2025-09-01' WHERE loan_id = 1")
conn.commit()

# Step 3: Return the book (this will create a fine)
return_info = return_book(1)
print(return_info)
# Example output: {'status': True, 'message': 'Book returned successfully', 'overdue_days': 25, 'fine': 250}

# Step 4: View unpaid fines for Member 1
unpaid = view_unpaid_fines(1)
print(unpaid)
# Example output:
# {'status': True, 'message': 'Unpaid fines found',
#  'fines': [{'loan_id': 1, 'due_date': '2025-09-01', 'return_date': '2025-09-26',
#             'fine_due': 250, 'paid': 0, 'balance': 250}]}


{'status': False, 'message': 'Loan ID not found'}
{'status': True, 'message': 'No unpaid fines', 'fines': []}
